In [33]:
# Learning notes
# 1. Chromedriver is located in "C:\Windows". When you need to update the driver, replace the .exe file in there.

## Prerequisites

In [34]:
import math
import pandas as pd

import plotly.subplots as sp
import plotly.graph_objects as go
import re

import time
import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup

import os
import glob
pause_time = 3

In [35]:
# Function to dissect the user_activity string
def dissect_user_activity(activity):
    # Initialize default values
    is_local_guide = False
    n_user_reviews = 0
    n_user_photos = 0

    # Check for 'Local Guide'
    if 'Local Guide' in activity:
        is_local_guide = True

    # Extract the number of reviews
    reviews_match = re.search(r'(\d+)\s+reviews?', activity)
    if reviews_match:
        n_user_reviews = int(reviews_match.group(1))

    # Extract the number of photos
    photos_match = re.search(r'(\d+)\s+photos?', activity)
    if photos_match:
        n_user_photos = int(photos_match.group(1))

    return pd.Series([is_local_guide, n_user_reviews, n_user_photos])



## Scrape Reviews

In [36]:
# estepalace: 'https://www.google.com/maps/place/estepalace+Hair+Transplant+Center/@41.0681014,28.9914843,17z/data=!3m1!4b1!4m6!3m5!1s0x14cab820372c2ef3:0x711c752f40eda6ca!8m2!3d41.0681014!4d28.993673!16s%2Fg%2F1tjj8mtd'
# estetistanbul: 'https://www.google.com/maps/place/Estet+Istanbul+Saglik+%26+Estetik/@41.0838758,29.011377,17z/data=!3m1!4b1!4m6!3m5!1s0x14cab66f08308d5b:0x87a8dbfee5c3f4b8!8m2!3d41.0838758!4d29.0135657!16s%2Fg%2F1tctn_f4'
# regina: 'https://www.google.com/maps/place/Regina+Med+Clinic+-+Sa%C3%A7+Ekimi+(Hair+Transplantation)+ve+Medikal+Estetik/@41.0557593,28.9961466,17z/data=!3m1!4b1!4m6!3m5!1s0x14cab71b379cabfd:0x33b9891e51c1350a!8m2!3d41.0557593!4d28.9987215!16s%2Fg%2F11t320c01n'

In [37]:
driver = webdriver.Chrome()  # Optional argument, if not specified will search path.
driver.implicitly_wait(pause_time)

df_name = "estepalace"
url = 'https://www.google.com/maps/place/estepalace+Hair+Transplant+Center/@41.0681014,28.9914843,17z/data=!3m1!4b1!4m6!3m5!1s0x14cab820372c2ef3:0x711c752f40eda6ca!8m2!3d41.0681014!4d28.993673!16s%2Fg%2F1tjj8mtd'
driver.get(url)
time.sleep(pause_time)

# Accept the cookies (ctrl q)
driver.find_element(By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/div[1]/form[2]/div/div/button/span').click()
time.sleep(pause_time)



In [38]:
#Find the total number of ratings
total_number_of_ratings = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span[2]/span[1]/span').text.split(" ")[0]
total_number_of_ratings = total_number_of_ratings.replace("(","")
total_number_of_ratings = int(total_number_of_ratings.replace(")",""))
print(total_number_of_ratings)

157


In [39]:
# Click on reviews and lay them out
xpath_str = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]/div[2]/div[2]'
driver.find_element(By.XPATH, xpath_str).click()

In [40]:
# # Alternative way to scroll down the review window
# html = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')
# html.send_keys(Keys.END)

# Get scroll height
# last_height = driver.execute_script("return document.body.scrollHeight")

n_loaded_objects = int(len(driver.find_elements(By.CLASS_NAME, 'jftiEf')))
ele = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')

# Scroll the page down to the bottom until there are no more reviews to load
while n_loaded_objects < int(total_number_of_ratings)-1:
    driver.execute_script('arguments[0].scrollBy(0, 5000);', ele)
    time.sleep(pause_time)
    n_loaded_objects = int(len(driver.find_elements(By.CLASS_NAME, 'jftiEf')))
    print(n_loaded_objects)



20
30
40
50
60
70
80
90
100
110
120
130
140
150
157


In [41]:
elems = driver.find_elements(By.CLASS_NAME, 'kyuRq.fontTitleSmall.WOKzJe') 
# By clicking on the "See original" buttons, each review is reverted back to its original language
for elem in elems:
        if 'See original' in elem.text:
                try:
                    elem.click()
                except:
                       continue
# <button class="kyuRq fontTitleSmall WOKzJe" role="switch" aria-checked="true" aria-controls="ChZDSUhNMG9nS0VJQ0FnSURRcVpUdkVBEAE" data-review-id="ChZDSUhNMG9nS0VJQ0FnSURRcVpUdkVBEAE" jsaction="pane.review.showReviewInOriginal" jslog="170842; track:click; mutable:true;metadata:WyIwYWhVS0V3aXhxZF9PNWE2QkF4WFdpdjBISGN5ekFzWVE0UjRJTVNnSiJd"> <span><span class="fontBodySmall iUHfzf">Translated by Google</span> <span class="fontBodySmall iUHfzf">・</span></span> <span>See original (Turkish)</span></button>

In [42]:
elems = driver.find_elements(By.CLASS_NAME, 'w8nwRe.kyuRq') 
# By clicking on the "More" buttons, each review that is too long to be fully displayed, is extended to its full format.
for elem in elems:
        if elem.text == "More":
                try:
                    elem.click()
                except:
                       continue


In [43]:
# Find all review elements just once
review_elements = driver.find_elements(By.CLASS_NAME, 'jftiEf')

# Initialize lists to store review data
reviews_data = []

# Loop through each review element to extract information
for review_element in review_elements:
    review_data = {}
    
    # Extract user ID and rating ID
    user_info_element = review_element.find_element(By.CLASS_NAME, 'WEBjve')
    review_data['user_id'] = user_info_element.get_attribute('data-href')
    review_data['rating_id'] = user_info_element.get_attribute('data-review-id')
    
    # Extract rating date and star rating
    rating_date_element = review_element.find_element(By.CLASS_NAME, 'rsqaWe')
    review_data['rating_date'] = rating_date_element.text
    
    star_rating_element = review_element.find_element(By.CLASS_NAME, 'kvMYJc')
    review_data['star_rating'] = star_rating_element.get_attribute('aria-label')
    
    # Attempt to extract review text and review language
    try:
        review_text_element = review_element.find_element(By.CLASS_NAME, 'MyEned')
        review_data['review'] = review_text_element.text
        review_data['review_lang'] = review_text_element.get_attribute('lang')
    except:
        review_data['review'] = ''
        review_data['review_lang'] = ''
    
    # Attempt to extract the user activity data if present
    try:
        user_activity_element = review_element.find_elements(By.CLASS_NAME, 'RfnDt')
        user_activity_element = user_activity_element[0]  # Assuming first element is the user activity data
        review_data['user_activity'] = user_activity_element.text
    except:
        review_data['user_activity'] = ''
    
    # Attempt to extract response details if present
    try:
        response_elements = review_element.find_elements(By.CLASS_NAME, 'DZSIDd')
        response_date_element = response_elements[0]  # Assuming first element is the response date
        review_data['response_date'] = response_date_element.text if response_date_element else ''
        
        response_text_elements = review_element.find_elements(By.CLASS_NAME, 'wiI7pd')
        response_text_element = response_text_elements[-1]  # Assuming the last element is the response text
        review_data['response'] = response_text_element.text if response_text_element else ''
        review_data['response_lang'] = response_text_element.get_attribute('lang') if response_text_element else ''
    except:
        review_data['response_date'] = ''
        review_data['response'] = ''
        review_data['response_lang'] = ''
    
    # Add the collected data to the list
    reviews_data.append(review_data)

# # Example: Print out the collected review data
# for data in reviews_data:
#     print(data)


In [44]:
df = pd.DataFrame(reviews_data)

In [45]:
rating_date_tf = []

# Get today's date
today = datetime.date.today()


for raw_rating in df.rating_date:
    units = int(1 if raw_rating.split(' ')[0] == 'a' else raw_rating.split(' ')[0])

    if 'year' in raw_rating:
        factor = 365
    elif 'month' in raw_rating:
        factor = 30
    elif 'week' in raw_rating:
        factor = 7
    elif 'day' in raw_rating:
        factor = 1
    
    approx_days = factor*units
    approx_date = today - datetime.timedelta(days=approx_days)
    rating_date_tf.append(approx_date)

df['rating_date_approx'] = rating_date_tf
df['rating_date_approx'] = pd.to_datetime(df['rating_date_approx'])

In [46]:
response_date_tf = []

# Get today's date
today = datetime.date.today()


for raw_rating in df.response_date:
    try:
        units = int(1 if raw_rating.split(' ')[0] == 'a' else raw_rating.split(' ')[0])

        if 'year' in raw_rating:
            factor = 365
        elif 'month' in raw_rating:
            factor = 30
        elif 'week' in raw_rating:
            factor = 7
        elif 'day' in raw_rating:
            factor = 1
        
        approx_days = factor*units
        approx_date = today - datetime.timedelta(days=approx_days)
        response_date_tf.append(approx_date)
    except:
        response_date_tf.append('')

df['response_date_approx'] = response_date_tf
df['response_date_approx'] = pd.to_datetime(df['response_date_approx'])


In [47]:
df['star_rating'] = [int(x[:1]) for x in df['star_rating']]

In [48]:
df['review'] = [x.replace('\n', ' ') for x in df['review']]
df['response'] = [x.replace('\n', ' ') for x in df['response']]

In [49]:
# Apply the function to create new columns
df[['is_local_guide', 'n_user_reviews', 'n_user_photos']] = df['user_activity'].apply(dissect_user_activity)

## Export

In [50]:
# df = pd.read_pickle('regina.pkl'.format(df_name))
df.to_pickle('{}.pkl'.format(df_name))

In [51]:
# df = pd.read_pickle('estetistanbul.pkl')